In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\text_summarization'

In [12]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_epochs: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [13]:
from text_summariser.constants import *
from text_summariser.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])



        model_trainer_config = ModelTrainerConfig(
            root_dir= Path,
            data_path= Path,
            model_ckpt= Path,
            num_train_epochs= int,
            warmup_epochs= int,
            per_device_train_batch_size= int,
            weight_decay= float,
            logging_steps= int,
            evaluation_strategy= str,
            eval_steps= int,
            save_steps= float,
            gradient_accumulation_steps= int
        )

        return model_trainer_config




In [17]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [18]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [19]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-06-09 13:08:47,298] [INFO] [common.py:18] [text_summariser.logging] - Read the yaml file from config\config.yaml loaded succesfully
[2024-06-09 13:08:47,306] [INFO] [common.py:18] [text_summariser.logging] - Read the yaml file from params.yaml loaded succesfully
[2024-06-09 13:08:47,310] [INFO] [common.py:34] [text_summariser.logging] - Created directory: artifacts
[2024-06-09 13:08:47,314] [INFO] [common.py:34] [text_summariser.logging] - Created directory: artifacts/model_trainer


OSError: Incorrect path_or_model_id: '<class 'pathlib.Path'>'. Please provide either the path to a local folder or the repo_id of a model on the Hub.